<a href="https://colab.research.google.com/github/sudkvij/GenAI/blob/main/RAG/Langchain1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LLM Chain

In [ ]:
#from langchain_openai import ChatOpenAI
#lim = ChatOpenAI(openai_api_key="")
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [ ]:
llm.invoke("how can langsmith help with testing?")

'\nLangsmith is a tool that can help with testing in several ways:\n\n1. **Code coverage analysis**: Langsmith can analyze the test code and provide insights into the code coverage, which can help identify areas of the code that are not being tested enough or at all.\n2. **Test case generation**: Langsmith can generate test cases based on the existing codebase, which can help developers write more comprehensive tests and reduce the number of false negatives.\n3. **Regression testing**: Langsmith can be used to identify changes in the codebase that may affect the functionality of the application, allowing developers to prioritize regression testing efforts.\n4. **Test data generation**: Langsmith can generate test data based on the existing data, which can help developers write more comprehensive tests and reduce the number of false negatives.\n5. **Test automation**: Langsmith can be used to automate testing tasks, such as running tests, analyzing test results, and identifying defects.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [ ]:
chain = prompt | llm

In [ ]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAh, an excellent question! As a world-class technical documentation writer, I must say that Langsmith is a versatile tool that can greatly assist in the testing process. Here are some ways Langsmith can help:\n\n1. Automated Testing: Langsmith's AI-powered engine can automatically generate test cases based on your technical documentation. This can save you time and resources by reducing the need for manual testing, allowing your team to focus on other critical tasks.\n2. Consistency Checking: Langsmith's advanced algorithms can analyze your documentation to identify inconsistencies and errors. By catching these issues early on, you can avoid potential problems down the line and ensure that your technical documents are accurate and up-to-date.\n3. Content Completeness Analysis: Langsmith can evaluate the completeness of your technical content by analyzing factors such as the inclusion of all necessary information, the clarity of explanations, and the accuracy of references. This can 

In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I'm glad you asked! Langsmith is more than just a language model - it's a versatile tool that can assist in various aspects of software testing, including documentation. Here are some ways Langsmith can help with testing:\n\n1. Automated Documentation Generation: Langsmith can automatically generate technical documentation for your software, saving you time and effort. With the ability to process large amounts of information, Langsmith can quickly create comprehensive documentation, including API references, user manuals, and more.\n2. Consistency and Accuracy: Langsmith's advanced language models ensure that your documentation is consistent and accurate across different parts of your software. This can help reduce errors and improve the overall quality of your documentation.\n3. Customization: Langsmith allows you to customize your documentation to fit your specific needs. You can create tailored documentation for different audiences

Retrieval Chain

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings()

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
<context>
{context}
</context>
Question: {input}""")
document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain_core.documents import Document
document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Based on the provided context, langsmith can help with testing by allowing users to visualize their test results. This means that users can use langsmith to create visual representations of their test data, such as graphs or charts, in order to better understand and analyze their test results. This can be particularly useful for identifying trends or patterns in the data, and for making informed decisions based on the test results.'

In [ ]:
from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing in several ways:

1. **Evaluating prompts**: LangSmith provides automatic evaluation metrics for LLMs, which can help identify areas where the prompt needs to be improved.
2. **Testing chains**: LangSmith allows you to test changes to a chain by running it over a dataset and visualizing the outputs. This helps identify any issues with the chain and make adjustments as needed.
3. **Collecting examples**: LangSmith provides an "Add to Dataset" button for each run, allowing you to easily collect examples of inputs and outputs for testing purposes.
4. **Collaborative debugging**: LangSmith enables collaboration in debugging by providing a "Share" button that allows you to share the chain and LLM runs with colleagues for debugging.
5. **Tracing**: LangSmith's tracing feature helps identify the sequence of calls and inputs/outputs of each call, which can help identify potential issues in the chain.
6. **Monitoring failures**: LangSmith allows you to monitor fa

Conversation Retrieval chain

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='inputs, and see what happens. At some point though, our application is performing\nwell and we want to be more rigorous about testing changes. We can use a dataset\nthat weâ€™ve constructed along the way (see above). Alternatively, we could spend some\ntime constructing a small dataset by hand. For these situations, LangSmith simplifies\ndataset uploading.Once we have a dataset, how can we use it to test changes to a prompt or chain? The most basic approach is to run the chain over the data points and visualize the outputs. Despite technological advancements, there still is no substitute for looking at outputs by eye. Currently, running the chain over the data points needs to be done client-side. The LangSmith client makes it easy to pull down a dataset and then run a chain over them, logging the results to a new project associated with the dataset. From there, you can review them. We\'ve made it easy to assign feedback to runs and mark them as correct or incorr